In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
from tqdm import tqdm
import os
import data_utils
import model_utils
from attack_utils import get_CSMIA_case_by_case_results, CSMIA_attack, LOMIA_attack
from data_utils import oneHotCatVars, filter_random_data_by_conf_score
from vulnerability_score_utils import get_vulnerability_score, draw_hist_plot
from experiment_utils import MIAExperiment
from disparity_inference_utils import get_confidence_array, draw_confidence_array_scatter, get_indices_by_group_condition, get_corr_btn_sens_and_out_per_subgroup, get_slopes, get_angular_difference, calculate_stds, get_mutual_info_btn_sens_and_out_per_subgroup
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network._base import ACTIVATIONS
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate
import pickle
# import utils
import copy

import matplotlib as mpl

# Setting the font family, size, and weight globally
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.weight'] = 'light'

In [5]:
i = -0.4
j = -0.4
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'correlation': 0,
            'subgroup_col_name': 'SEX',
            'marginal_prior': 1,
            'corr_btn_sens_and_output_per_subgroup': (i, j),
            # 'fixed_corr_in_test_data': True
    }, shortname = f"Corr_btn_sens_and_output_for_male_({i})_for_female_({j})"
)

[0, 1]
{0: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}, 1: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}}


100%|██████████| 2/2 [00:01<00:00,  1.69it/s]


{0: {(0, 1): 6875, (0, 0): 5625, (1, 1): 5625, (1, 0): 6875}, 1: {(0, 1): 7500, (0, 0): 5000, (1, 1): 5000, (1, 0): 7500}}


100%|██████████| 2/2 [00:01<00:00,  1.75it/s]


In [6]:
experiment.original_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==0]
experiment.original_df = experiment.original_df.drop(['is_train'], axis=1)
experiment.aux_df = experiment.ds.ds.original_df[experiment.ds.ds.original_df['is_train']==1]
experiment.aux_df = experiment.aux_df.drop(['is_train'], axis=1)
experiment.y_column = experiment.ds.ds.meta['y_column']

In [7]:
# sens_pred, case_indices = CSMIA_attack(experiment.clf_only_on_test, experiment.X_test, experiment.y_te, experiment.ds.ds.meta)
# experiment.correct_indices = (sens_pred == experiment.X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())
# experiment.incorrect_indices = ~experiment.correct_indices
experiment.sensitive_column = experiment.ds.ds.meta["sensitive_column"]
experiment.sens_ground_truth = experiment.X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel()

In [8]:
try:
    experiment.all_vuln_scores = np.load(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_vuln_scores.npy')
except:
    experiment.all_vuln_scores = np.array([get_vulnerability_score(experiment, experiment.X_test, experiment.y_te, experiment.original_df, index, k=4) for index in tqdm(experiment.X_test.index)])
    np.save(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_vuln_scores.npy', experiment.all_vuln_scores)
experiment.all_vuln_scores_rounded = np.round(experiment.all_vuln_scores)
# experiment.vuln_accuracy = accuracy_score(experiment.correct_indices, experiment.all_vuln_scores_rounded)

In [9]:
X_test_w_vuln = experiment.X_test.copy()
# X_test_w_vuln[['vuln']] 
X_test_w_vuln['vuln'] = pd.Series(experiment.all_vuln_scores_rounded, index=X_test_w_vuln.index)

In [10]:
dataset = torch.utils.data.TensorDataset(torch.tensor(X_test_w_vuln.values).float(), torch.tensor(experiment.y_te_onehot).float())
train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

In [11]:
X_temp = experiment.X_test.copy().reset_index(drop=True)
# X_test_w_vuln[['vuln']] 
X_temp['vuln'] = pd.Series(experiment.all_vuln_scores_rounded, index=X_temp.index)

vuln_index = X_temp[X_temp['vuln']==1].index

vuln_dataset = torch.utils.data.TensorDataset(torch.tensor(X_temp.loc[vuln_index].values).float(), torch.tensor(experiment.y_te_onehot[vuln_index]).float())
vuln_train_loader = torch.utils.data.DataLoader(vuln_dataset, batch_size=128, shuffle=True)

non_vuln_index = X_temp[X_temp['vuln']==0].index

non_vuln_dataset = torch.utils.data.TensorDataset(torch.tensor(X_temp.loc[non_vuln_index].values).float(), torch.tensor(experiment.y_te_onehot[non_vuln_index]).float())
non_vuln_train_loader = torch.utils.data.DataLoader(non_vuln_dataset, batch_size=128, shuffle=True)

In [22]:
class PortedMLPClassifier(nn.Module):
    def __init__(self, n_in_features=37, n_out_features=2):
        super(PortedMLPClassifier, self).__init__()
        layers = [
            nn.Linear(in_features=n_in_features, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=16),
            nn.ReLU(),
            nn.Linear(in_features=16, out_features=8),
            nn.ReLU(),
            nn.Linear(in_features=8, out_features=n_out_features),
            nn.Softmax(dim=1)
        ]
        self.layers = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor):
        return self.layers(x)
    
    def predict_proba(self, x: torch.Tensor):
        return self.forward(x)
    
def test_model(model, X_test, y_te_onehot):
    x_te = X_test.values
    dataset = torch.utils.data.TensorDataset(torch.tensor(x_te).float(), torch.tensor(y_te_onehot).float())
    test_loader = torch.utils.data.DataLoader(dataset, batch_size=x_te.shape[0], shuffle=False)

    model.eval()
    y_pred = []
    y_true = []
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = data.to('mps'), target.to('mps')
        output = model(data)
        y_pred.append(output.cpu().detach().numpy())
        y_true.append(target.cpu().detach().numpy())

    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    y_pred = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_true, axis=1)

    return accuracy_score(y_true, y_pred)
    return(classification_report(y_true, y_pred))

In [13]:
base_model = model_utils.get_model()
base_model.partial_fit(np.zeros_like(experiment.X_test.values), np.zeros_like(experiment.y_te_onehot), classes=np.unique(experiment.y_te))
model = model_utils.port_mlp_to_ch(base_model)

In [14]:
max_grad_norm = 1.0  # Maximum norm for gradient clipping
noise_multiplier = 1.0  # Noise multiplier to control the level of privacy

# Function to clip gradients
def clip_gradients(parameters, max_grad_norm):
    total_norm = 0.0
    # Calculate total gradient norm for each parameter
    for param in parameters:
        if param.grad is not None:
            total_norm += param.grad.data.norm(2).item() ** 2
    total_norm = total_norm ** 0.5  # L2 norm of the gradients

    # Clip gradients to have a maximum norm
    clip_coef = max_grad_norm / (total_norm + 1e-6)
    if clip_coef < 1:
        for param in parameters:
            if param.grad is not None:
                param.grad.data.mul_(clip_coef)

# Function to add noise to gradients
def add_noise_to_gradients(parameters, noise_multiplier, max_grad_norm, device):
    for param in parameters:
        if param.grad is not None:
            # Add Gaussian noise to the gradient
            # Print the gradient before adding noise for debugging
            # print(f"Before adding noise: {param.grad}")
            
            # Generate Gaussian noise
            noise = torch.normal(mean=0, std=noise_multiplier * max_grad_norm, size=param.grad.shape).to(device)
            
            # Print the generated noise for debugging
            # print(f"Noise to add: {noise}")
            
            # Add noise directly to the gradient
            param.grad.add_(noise)
            
            # Print the gradient after adding noise for debugging
            # print(f"After adding noise: {param.grad}")

In [15]:
model = PortedMLPClassifier(n_in_features=experiment.X_train.shape[1], n_out_features=experiment.y_tr_onehot.shape[1]).to('mps')

In [16]:
model = PortedMLPClassifier(n_in_features=experiment.X_train.shape[1], n_out_features=experiment.y_tr_onehot.shape[1]).to('mps')

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in tqdm(range(10)):
    for batch_idx, (data, target) in enumerate(train_loader):
        model.train()
        optimizer.zero_grad()
        data, target = data.to('mps'), target.to('mps')
        output = model(data[:, :-1])
        loss = nn.BCELoss()(output, target)
        loss.backward()
        if batch_idx % 2 == 0 or True:  # Apply DP to half of the batches (for example)
            # Step 1: Clip gradients
            clip_gradients(model.parameters(), max_grad_norm)

            # Step 2: Add noise to gradients
            add_noise_to_gradients(model.parameters(), noise_multiplier, max_grad_norm, device='mps')
        optimizer.step()
        # break

100%|██████████| 10/10 [00:42<00:00,  4.27s/it]


In [19]:
def get_selective_dp_model(noise_multiplier=1.0):
    model = PortedMLPClassifier(n_in_features=experiment.X_train.shape[1], n_out_features=experiment.y_tr_onehot.shape[1]).to('mps')

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    for epoch in tqdm(range(10)):
        for batch_idx, (data, target) in enumerate(non_vuln_train_loader):
            model.train()
            optimizer.zero_grad()
            data, target = data.to('mps'), target.to('mps')
            output = model(data[:, :-1])
            loss = nn.BCELoss()(output, target)
            loss.backward()
            # if batch_idx % 2 == 0 or True:  # Apply DP to half of the batches (for example)
            #     # Step 1: Clip gradients
            #     clip_gradients(model.parameters(), max_grad_norm)

            #     # Step 2: Add noise to gradients
            #     add_noise_to_gradients(model.parameters(), noise_multiplier, max_grad_norm, device='mps')
            optimizer.step()
            # break
        for batch_idx, (data, target) in enumerate(vuln_train_loader):
            model.train()
            optimizer.zero_grad()
            data, target = data.to('mps'), target.to('mps')
            output = model(data[:, :-1])
            loss = nn.BCELoss()(output, target)
            loss.backward()
            if batch_idx % 2 == 0 or True:  # Apply DP to half of the batches (for example)
                # Step 1: Clip gradients
                clip_gradients(model.parameters(), max_grad_norm)

                # Step 2: Add noise to gradients
                add_noise_to_gradients(model.parameters(), noise_multiplier, max_grad_norm, device='mps')
            optimizer.step()

    return model

In [26]:
sigmas = [0.1, 0.25, 0.5, 1, 2, 3, 5, 10]

model_by_sigmas = {}

for sigma in sigmas:
    try:
        model = PortedMLPClassifier(n_in_features=experiment.X_train.shape[1], n_out_features=experiment.y_tr_onehot.shape[1]).to('mps')

        model.load_state_dict(torch.load(f"{experiment.ds.ds.filenameroot}_selective_dp_{sigma}.pt"))
    except:
        model = get_selective_dp_model(noise_multiplier=sigma)
        torch.save(model.state_dict(), f"{experiment.ds.ds.filenameroot}_selective_dp_{sigma}.pt")

    model_by_sigmas[sigma] = model    

100%|██████████| 10/10 [00:34<00:00,  3.48s/it]


In [27]:
for sigma in model_by_sigmas:
    print(f'Sigma: {sigma}')
    model = model_by_sigmas[sigma]
    test_acc = test_model(model, experiment.X_train, experiment.y_tr_onehot)
    print(f' Test Accuracy: {test_acc}')
    csmia_acc = get_CSMIA_case_by_case_results(model, experiment.X_test, experiment.y_te, experiment.ds, 'SEX', metric='accuracy', sensitive_col_name=None).loc['Case All Cases', 'Overall']
    print(f' CSMIA Accuracy: {csmia_acc}')

Sigma: 0.1
 Test Accuracy: 0.65358
 CSMIA Accuracy: 70.0
Sigma: 0.25
 Test Accuracy: 0.575
 CSMIA Accuracy: 70.0
Sigma: 0.5
 Test Accuracy: 0.50818
 CSMIA Accuracy: 66.486
Sigma: 1
 Test Accuracy: 0.5753
 CSMIA Accuracy: 69.936
Sigma: 2
 Test Accuracy: 0.58466
 CSMIA Accuracy: 69.342
Sigma: 3
 Test Accuracy: 0.4825
 CSMIA Accuracy: 30.0
Sigma: 5
 Test Accuracy: 0.53378
 CSMIA Accuracy: 59.036
Sigma: 10
 Test Accuracy: 0.57606
 CSMIA Accuracy: 30.0


In [1]:
csmia_acc.loc['Case All Cases', 'Overall']

NameError: name 'csmia_acc' is not defined

In [49]:
get_CSMIA_case_by_case_results(model, experiment.X_test, experiment.y_te, experiment.ds, 'SEX', metric='accuracy', sensitive_col_name=None)

,1,0,Overall
Case 1,24789 (69.9625),24799 (69.9665),69.9645
Case 2,169 (76.3314),131 (78.626),77.3333
Case 3,42 (64.2857),70 (68.5714),66.9643
Case All Cases,25000 (69.996),25000 (70.008),70.0020
